# Interacción con archivos de Internet

Veremos en este cuaderno diferentes formas de  interactuar con páginas o sitios de Internet. Como vimos en el cuaderno pasado estas interaciones se rigen por el protocolo HTML.

## 1. Estructura básica de los documentos html

- Cada página comienza con `<html>`.
- A continuación viene la cabecera, delimitada por `<head>` y `</head>`.
- Después, el comando `<body>`, que indica el comienzo del cuerpo de la página. 
- Las el contenido de la página y las instrucciones HTML se escribirán a continuación, y finalizarán con `</body>`.
- La página acabará con `</html>`.
- Todo elemento HTML  comienza con un tag tipo `<tag>` y  cierra en el mismo tag  precedido de `/`,  es decir `</tag>`. 

*Ejemplo.* La siguiente es una página HTML válida. `title` es el tag para el título de la página (el que aparece en la tab del navegador)  `h1` es el tag para un encabezamiento de tamaño grande y `p` es el tag correspondiente a un párrafo. 
```
<html>

<head>
    <title>Título de la página</title>
</head>

<body>
    <h1>Un encabezamiento</h1>
    <p>Un parágrafo.</p>
    <p>Esté es otro párrafo.</p>
</body>

</html>
```
Las página permiten mútiples tipos de elementos,  entre ellos *tablas* que es una entidad que usaremos luego. 

*Ejemplo.* El siguiente es código HTML válido con una tabla incluida. 


```
<html>
<head>
<title>Page Title</title>
</head>
<body>
<h1>This is a Heading</h1>
<p>This is a paragraph.</p>
<p>A continuación una tabla.</p>
<p>
    <table>
        <tr>
            <td>a</td> <td>b</td> <td>c</td> 
        </tr>
        <tr>
            <td>1234</td> <td>50</td> <td>ddd</td>
        </tr>
    </table>
</p>
</body>
</html>
```



## 2. Uso de `BeautifulSoup` para estructurar páginas HTML

Recordemos que para bajar archivos HTML usámos el  módulo `requests`. A continuación lo incorporamos:

In [ ]:
import requests

Accedamos al sitio español [www.tutiempo.net](https://www.tutiempo.net) que contiene diversa información meteorológica,  incluyendo mucha información histórica de diversa partes del mundo, incluyendo argentina.

Por ejemplo,  si queremos información el día 5 de marzo de 2019  en Córdoba,  más precisamente en el aeropuerto, cargamos la página:

 [https://www.tutiempo.net/registros/saco/5-marzo-2019.html](https://www.tutiempo.net/registros/saco/5-marzo-2019.html)

 También la podemos bajar usando el módulo `requests`

In [ ]:
respuesta = requests.get('https://www.tutiempo.net/registros/saco/5-marzo-2019.html')
type(respuesta)

In [ ]:
print(respuesta.text)

Como podrán observar, el código interno de la página no es de fácil lectura. En  lo que sigue usaremos `BeautifulSoup` para extraer información útil de la página. 

En  realidad  `BeautifulSoup` es un submódulo de `bs4` y lo importaremos como tal. También debemos instalar `lxml` que es un módulo utilizado por  `BeautifulSoup` para analizar (*parsear* en la jerga computacional) la página. 



In [ ]:
# pip install beautifulsoup4
# pip install lxml
# Ya están instalados en Colab. En un entorno propio deben instalarse manualmente.
from bs4 import BeautifulSoup

Recordemos que con `requests` creamos el objeto `respuesta` que nos permite acceder a los datos de la página web  [https://www.tutiempo.net/registros/saco/5-marzo-2019.html](https://www.tutiempo.net/registros/saco/5-marzo-2019.html)

Ahora creamos un objeto `BeautifulSoup` a partir de lo obtenido: 

In [ ]:
soup = BeautifulSoup(respuesta.content, "lxml")
print(type(soup))

Para ver mejor los contenidos  del archivo pordemos usar el  método `prettify`:

In [ ]:
print(type(soup.prettify()))
print(soup.prettify())

Esto nos permite ver más claramente la estructura del archivo y encontrar las tags que nos interesan. 

No vamos a explicar aquí las capacidades de `BeautifulSoup`, simplemente daremos algunos ejemplos. El  estudiante interesado buscará algún tutorial sobre el tema (hay cientos de tutoriales sobre `BeautifulSoup`).



Primero extraigamos los registros del día. Inspeccionando el documento vemos que hay una única tabla y esta tabla tiene los registros hora por hora. Un  código para extraer estos registros de un día determinado (en Córdoba) podría ser:

In [ ]:
def tabla_del_dia(dia, mes, año):
    nombre_fecha = str(dia) + '-' + mes + '-' + str(año)
    contenido_url = requests.get('https://www.tutiempo.net/registros/saco/' + nombre_fecha + '.html')
    soup = BeautifulSoup(contenido_url.text, 'lxml') # parsea la página 
    tabla_dia = soup.find('table', {'style': 'width: 100%'}) # extrae la tabla (es la única con style="width: 100%")
    return tabla_dia # devuelve la tabla

print(tabla_del_dia(29, 'mayo', 2021).prettify() )

Finalmente, de la tabla podemos extraer el valor de las celdas: 

In [ ]:
# Basado en: 1) https://stackoverflow.com/questions/17196018/extracting-table-contents-from-html-with-python-and-beautifulsoup
# 2) https://www.kite.com/python/examples/4420/beautifulsoup-parse-an-html-table-and-write-to-a-csv

# La información la extraemos de la estación metereológica SACO, correspondiente al Aeropuerto de Córdoba. 

def registro_fila_tabla(fila_tabla) -> list: # procesa una fila de la tabla, devuelve la lista con las mediciones registradas en esa fila
    celdas_fila = fila_tabla.findAll('td')
    registro = []
    for celda in celdas_fila:                # recorre las celdas de la fila correspondiente a una toma de mediciones
        registro.append(celda.text)
    return registro

def registros_tabla(tabla) -> list: # procesa una tabla, devuelve la lista de registros contenidos en esa tabla
    registros = []
    for fila_tabla in tabla.findAll('tr'):
        registro = registro_fila_tabla(fila_tabla)
        if len(registro) != 0:
            registros.append(registro)
    return registros

def registros_dia(dia, mes, año) -> list:
    tabla_dia =tabla_del_dia(dia, mes, año)
    return registros_tabla(tabla_dia)


# Ejemplo 
fecha_proc = registros_dia(29, 'mayo', 2021) 
print(type(fecha_proc))
print(type(fecha_proc[0]))
for w in fecha_proc:
    print(w)

Algunos sitios restringen el  acceso por `requests` y  el texto que se devuelve no tiene contenido útil. Esto se soluciona haciendo un truco: 

In [ ]:
url_base = 'https://www.promiedos.com.ar/verfecha.php?fecha='
num_fecha =  '5'
num_camp = '14'
fecha = url_base + '"' + str(num_fecha) + '_' + str(num_camp) + '"' # es la url donde está la fecha
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
res = requests.get(fecha, headers=headers)
soup = BeautifulSoup(res.content, "lxml")

print(soup.prettify)

## 3. Lectura y escritura de archivos JSON

JSON, cuyo nombre corresponde a las siglas *JavaScript Object Notation* o *Notación de Objetos de JavaScript*, es un formato ligero de intercambio de datos, que resulta sencillo de leer y escribir para los programadores y simple de interpretar y generar para las máquinas.

JSON es un formato de texto completamente independiente de lenguaje y puede ser interpretado y creado por los siguientes lenguajes:

- C
- C++
- C#
- Java
- JavaScript
- Perl
- Python
- etc. 

Muchos lenguajes de programación proporcionan métodos para analizar una cadena de texto con este formato en un objeto nativo y viceversa. 

Pese a su nombre, no es necesariamente parte de JavaScript, de hecho, es un estándar basado en texto plano para el intercambio de datos, por lo que se usa en muchos sistemas que requieren mostrar o enviar información para ser interpretada por otros sistemas.

Una de las características de JSON, al ser un formato que es independiente de cualquier lenguaje de programación, es que los servicios que comparten información por este método no necesitan hablar el mismo idioma, es decir, el emisor puede ser Java y el receptor Python, pues cada uno tiene su propia librería para codificar y decodificar cadenas en este formato. 



Veamos como leer y escribir archivos json:

In [ ]:
import json

with open('./sample_data/anscombe.json', 'r') as f:
    data = f.read() # data es una str

archivo = json.loads(data) # archivo es (en este caso) una lista de diccionarios
print(type(archivo))
print(type(archivo[0]))

print(archivo)

print(archivo[0]['X'])
    

for item in archivo:
    print(item['X']) 

Lo notable de JSON  es que los archivos de texto en formato JSON son muy parecidos a los diccionarios y listas de Python,  con una sintaxis idéntica. 

La operación inversa a `json.loads()` es `json.dumps()`:  transforma un diccionario o lista en una cadena con  estructura JSON.  

In [ ]:
import json 
u = {'a': {2 : 0 }, 'b': {1 : 5 }}
print (type(u), u)
print(type(str(u)), str(u))
x = json.dumps({'a': {2 : 0 }, 'b': {1 : 5 }})
print(type(x), x)
y = json.loads(x)
print(type(y), y)

# observar que el diccionario original cambia las claves numérica de TODOS los diccionarios a tipo str, incluso los diccionarios que son valores de otro diccionario.
# Los valores numéricos se mantienen numéricos. Si un valor es un lista,  se mantiene lista.

*Ejemplo* Tenemos una lista de diccionarios y queremos escribir uno por línea en un archivo, para que luego sea leido usando `json.loads()`. En  la celda de código siguiente hacemos un ejemplo. 

In [ ]:
import json 
import random

# Creamos una lista con 100 diccionarios tipo {'x' : entero al azar, 
# 'y' : entero al azar, nº de orden en la lista : [entero al azar, entero al azar] }
# Los guardaremos en el archivo prueba.txt, uno en cada linea. 

lista_dic = []
for i in range(100):
  x, y, w0, w1 = random.randint(1, 1000), random.randint(1, 1000), random.randint(1, 1000), random.randint(1, 1000)
  lista_dic.append({ 'x' : x, 'y' : y, i : [w0 , w1]})

# Guardamos en el archivo prueba.txt, línea por línea el diccionario.
f = open('prueba.txt', 'w')
for i in range(len(lista_dic)):
  f.write(json.dumps(lista_dic[i])+'\n') # \n pasa de renglón
f.close()

# Revisar el archivo prueba.txt

# Para leer debemos hacerlo también linea por linea, pues el archivo 
# no tiene la estructura lógica de un diccionario o lista 
# (debería abrir con { o [ y cerrar con } o  ],  respectivamente)

# Ahora haremos una lista de diccionarios a partir de prueba.txt

lista_dic2 = []
f = open('prueba.txt', 'r')
for linea in f:
  lista_dic2.append(json.loads(linea))
f.close()

print(type(lista_dic2[3]), lista_dic2[3]) # comprobamos que cada coordenada 
# es un diccionario

# Importante:  las claves numéricas pasaron a str. Los valores matienen su tipo (int, list, etc)
print(lista_dic[3][3]) 
print(lista_dic2[3]['3']) # No estaría bien  print(lista_dic2[3][3]) 


*Ejemplos de cosas que  **NO** funcionan.*


In [ ]:
import json
# Si hacemos json.dumps()  de una string, Python lo hace, pero no es el resultado que esperamos.
x = json.dumps("{'a': {2 : 0 }, 'b': {1 : 5 }")
print(type(x), x)
y = json.loads(x)
print(type(y), y) # json.loads() en este caso devuelve una string!
# y es una string que "parece" un diccionario. Pero no lo es. 
# z = json.loads(y) #  esto directamente da error


**Recomendación:** Usar `json.loads()` y `json.dumps()` de forma prolija y de acuerdo a los lineamientos escritos más arriba. Quizás haciendo aguna  "magia" con los `replace()` se pueda arreglar la celda de código anterior, pero es mejor no hacerlo.

## 4. Uso de la API de NOAA

La Oficina Nacional de Administración Oceánica y Atmosférica (National Oceanic and Atmospheric Administration, NOAA) es una agencia científica del Departamento de Comercio de los Estados Unidos cuyas actividades se centran en monitorear las condiciones de los océanos y la atmósfera. 

El NOAA ofrece distintos *datasets* (conjuntos de datos estructurados) sin limitaciones. Los datos puedes ser bajados directamente del sitio web https://www.noaa.gov/ o, alternativamente, accedidos por API. 

En esta sección accederemos a los datasets utilizando la API. Para utilizar la API se debe solicitar  un *token* al NOAA (ver *Referencias*). El token es una cadena de caracteres que funciona como id y password al mismo tiempo. 

Si al comienzo ponemos 

    import requests
    token = 'nuestro token'
    my_headers = {'token' : token}
    response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets', headers=my_headers)
    respuesta = response.json()
    resultados = respuesta['results'] # Datasets disponible
    for w in resultados:
      pass
      print(w['uid'],':',w['name'])

obtendremos el `id` de cada uno de los datasets ofrecidos por el NOAA via API y una breve descripción de los mismos. En particular,  son interesantes:

    GHCND : Daily Summaries
    GSOM : Global Summary of the Month
    GSOY : Global Summary of the Year

La API permite una variedad de consultas, por ejemplo

    response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets?locationid=CITY:US390029', headers=my_headers)
    respuesta = response.json()

nos devuelve todos los datasets disponibles para determinada localidad. 

También podemos obtener, por ejemplo, datos climatológicos de una determinada localidad durante un período determinado de tiempo: la consulta

    response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:AR000087344&units=metric&startdate=2021-05-01&enddate=2021-05-31', headers=my_headers)
    respuesta = response.json()
    resultados = respuesta['results']

nos devuelve un sumario de temperaturas y precipitaciones diarias en el mes de mayo de la estación meteorológica del Aeropuerto de Córdoba.  

En las referencias a continuación se encuentra como obtener el token, la descripción de la API, la lista de estaciones meteorológicas y mucha más información. Luego viene una celda de código con ejemplos. 



*Referencias*
- https://www.ncdc.noaa.gov/ National Centers for Environmental Information (37 petabytes)
- https://www.ncei.noaa.gov/support/access-data-service-api-user-documentation
- Para bajar estaciones meteorolgógicas:  https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
- https://www.ncdc.noaa.gov/ghcn-daily-description
- Para bajar manual de ghcn-daily: https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt
- Para pedir token: https://www.ncdc.noaa.gov/cdo-web/token
- Climate Data Online - Web Services Documentation: https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted
- Manual genérico para uso de APIs en Python: https://www.nylas.com/blog/use-python-requests-module-rest-apis/




In [ ]:
import requests
token = 'LsJXpYprYMzOKjMtqAlDmMUzwGelIQDf'
my_headers = {'token' : token}
# Ejemplo: datasets disponibles en NOAA
response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets', headers=my_headers)
respuesta = response.json()

resultados = respuesta['results'] # Datasets disponibles

for w in resultados:
  pass
  print(w['id'],':',w['name'])


In [ ]:
# Ejemplo: datasets  disponible en la ciudad US390029
response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets?locationid=CITY:US390029', headers=my_headers)
respuesta = response.json()
print(respuesta)


In [ ]:
# Datos climatológicos diarios (temperatura máxima, temperatura mínima y precipitaciones) en un período de tiempo en
# la estación meteorológica  AR000087344 (Aeropuerto de Córdoba) 
fecha_ini, fecha_fin = '2021-05-28', '2021-05-29'
response = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&stationid=GHCND:AR000087344&limit=1000&units=metric&startdate='+fecha_ini+'&enddate='+fecha_fin+'', headers=my_headers)
respuesta = response.json()
resultados = respuesta['results']
print(response.json())

In [ ]:
for w in resultados:
  print(w)
# Observación 1. Cuidado: no necesariamente todos los día tienen todos los datos.
# Observación 2. Por defecto la API  recupera 25 registros por llamada. En la llamada anterior pusimos 'limit=1000'
#                para recuperar hasta 1000 registros por llamada. La API no permite más. 

Observar que cada elemento de  `resultados` es un diccionario y en ese diccionario  estan las claves`date`, `datatype` y `value`. 
- `date` indica el día. 
- Si el `datatype` es `TMAX`,  entonces `value` indica la temperatura máxima del día.
- Si el `datatype` es `TMIN`,  entonces `value` indica la temperatura mínima del día.
- Si el `datatype` es `TAVG`,  entonces `value` indica la temperatura promedio del día.
- Si el `datatype` es `PRCP`,  entonces `value` indica la precipitación (en mm) de ese día. 